# LangGraph Tools: Currency Converter

## Objective
Build a practical currency conversion tool that demonstrates multi-parameter tools with validation and error handling.

## What You'll Learn
1. Creating tools with multiple parameters
2. Writing comprehensive docstrings with Args documentation
3. Implementing input validation in tools
4. Handling edge cases and errors gracefully
5. Returning formatted, human-readable results

## Prerequisites
- Completed: `02_getting_started_with_langgraph_tools.ipynb`
- Understanding of the `@tool` decorator and `.invoke()` pattern

---

## Section 1: Setup

Import the required dependencies.

In [ ]:
# Core imports
from langchain_core.tools import tool
from pprint import pprint

print("✅ Imports successful")

---

## Section 2: Understanding the Currency Converter Tool

### What This Tool Does
Converts amounts between five major currencies: USD, EUR, GBP, INR, and JPY.

### Design Decisions

| Aspect | Decision | Rationale |
|--------|----------|----------|
| **Exchange Rates** | Hardcoded (relative to USD) | Simplicity for demo; production would use an API |
| **Conversion Method** | Convert to USD first, then to target | Single reference point simplifies calculations |
| **Return Type** | Formatted string | Human-readable for LLM to relay to users |
| **Error Handling** | Return error messages as strings | Graceful failure without exceptions |

### Reference Point: Multi-Step Conversion Logic

```
Source Currency → USD (base) → Target Currency

Example: 1000 INR → USD → EUR
  Step 1: 1000 INR ÷ 83.12 = 12.03 USD
  Step 2: 12.03 USD × 0.92 = 11.07 EUR
```

---

## Section 3: Building the Currency Converter Tool

Notice these key elements:
1. **Three parameters** with clear type hints
2. **Detailed docstring** with Args section (LLM reads this!)
3. **Input validation** before processing
4. **Formatted output** with conversion details

In [ ]:
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between 
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    
    Args:
        amount: The amount to convert (must be positive)
        from_currency: Source currency code (USD, EUR, GBP, INR, JPY)
        to_currency: Target currency code (USD, EUR, GBP, INR, JPY)
    
    Returns:
        A string with the conversion result including the exchange rate
    """
    
    # Exchange rates relative to USD (as of demo date)
    # In production, fetch real-time rates from an API
    exchange_rates = {
        "USD": 1.0,
        "EUR": 0.92,
        "GBP": 0.79,
        "INR": 83.12,
        "JPY": 149.50
    }
    
    # Normalize currency codes to uppercase
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    # Validate source currency
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency '{from_currency}'. Supported: {', '.join(exchange_rates.keys())}"
    
    # Validate target currency
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency '{to_currency}'. Supported: {', '.join(exchange_rates.keys())}"
    
    # Convert: Source → USD → Target
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    
    # Calculate effective exchange rate for display
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    # Format result
    result = (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )
    
    return result

print("✅ currency_converter tool defined")
print(f"   Tool Name: {currency_converter.name}")
print(f"   Tool Type: {type(currency_converter)}")

### Reference Point: Docstring Best Practices

The docstring serves two purposes:

| Audience | What They Use |
|----------|---------------|
| **LLM** | First paragraph - decides when to call the tool |
| **LLM** | Args section - understands what values to provide |
| **Developers** | Full docstring - understands implementation |

> **Tip:** The first line of your docstring should clearly state WHEN to use the tool, not just what it does.

---

## Section 4: Testing the Currency Converter

Always test tools directly before integrating them into a graph. We'll cover:
1. Standard conversion (happy path)
2. Error handling (invalid currency)
3. Edge case (same currency conversion)

In [ ]:
print("=" * 70)
print("TEST CASE 1: Standard Conversion (USD → EUR)")
print("=" * 70)

result = currency_converter.invoke({
    "amount": 1000,
    "from_currency": "USD",
    "to_currency": "EUR"
})
print(result)

In [ ]:
print("=" * 70)
print("TEST CASE 2: Error Handling (Invalid Currency)")
print("=" * 70)

result = currency_converter.invoke({
    "amount": 100,
    "from_currency": "XYZ",
    "to_currency": "USD"
})
print(result)

In [ ]:
print("=" * 70)
print("TEST CASE 3: Edge Case (Same Currency)")
print("=" * 70)

result = currency_converter.invoke({
    "amount": 5000,
    "from_currency": "INR",
    "to_currency": "INR"
})
print(result)

In [ ]:
print("=" * 70)
print("TEST CASE 4: Case Insensitivity")
print("=" * 70)

result = currency_converter.invoke({
    "amount": 500,
    "from_currency": "gbp",  # lowercase
    "to_currency": "jpy"     # lowercase
})
print(result)

print("\n" + "=" * 70)
print("✅ All tests completed")
print("=" * 70)

### Reference Point: Testing Strategy

When testing tools, always cover:

| Test Type | Purpose | Example |
|-----------|---------|--------|
| **Happy Path** | Normal expected usage | USD → EUR conversion |
| **Error Cases** | Invalid inputs | Unsupported currency code |
| **Edge Cases** | Boundary conditions | Same currency, zero amount |
| **Input Variations** | Different valid formats | Lowercase currency codes |

---

## Section 5: Examining the Tool Schema

Let's see what the LLM sees when this tool is available. The schema is auto-generated from our function definition.

In [ ]:
print("Currency Converter - Tool Schema")
print("=" * 70)
print("\nThis is what the LLM sees:\n")

schema = currency_converter.args_schema.model_json_schema()
pprint(schema)

### Reference Point: Schema Analysis

Looking at the generated schema:

```json
{
  "properties": {
    "amount": {"type": "number"},        // float → number
    "from_currency": {"type": "string"}, // str → string  
    "to_currency": {"type": "string"}    // str → string
  },
  "required": ["amount", "from_currency", "to_currency"]
}
```

**Key Observations:**
1. All three parameters are marked as `required`
2. `float` becomes `number` in JSON schema
3. The description from docstring Args is included (if using Pydantic models)

> **Note:** For more control over descriptions, you can use Pydantic models with `Field()` - covered in advanced tutorials.

---

## Section 6: How the LLM Would Use This Tool

When integrated with an LLM in a LangGraph workflow, here's what happens:

### User Query → Tool Call Flow

```
User: "How much is 500 euros in Japanese yen?"
          ↓
LLM analyzes query and available tools
          ↓
LLM outputs AIMessage with tool_calls:
  {
    "name": "currency_converter",
    "args": {
      "amount": 500,
      "from_currency": "EUR",
      "to_currency": "JPY"
    }
  }
          ↓
ToolNode executes: currency_converter.invoke({...})
          ↓
Result returned to LLM
          ↓
LLM formats response for user
```

> **Remember:** The LLM doesn't execute the tool—it only requests the call. LangGraph's `ToolNode` handles actual execution.

---

## Summary

In this notebook, you learned:

| Concept | Key Takeaway |
|---------|-------------|
| **Multi-parameter tools** | Use clear type hints for each parameter |
| **Docstrings** | Include Args section for LLM understanding |
| **Input validation** | Validate early, return helpful error messages |
| **Error handling** | Return errors as strings, not exceptions |
| **Testing** | Cover happy path, errors, and edge cases |

## Next Steps

In the next notebook, we'll build the **EMI Calculator** tool which demonstrates:
- More complex mathematical calculations
- Four parameters with different types
- Formatted financial output

---

## Practice Exercises

Try extending the currency converter with these challenges:

1. **Add a new currency** - Add support for AUD (Australian Dollar)
2. **Add validation** - Return an error if the amount is negative
3. **Create a reverse tool** - Build a tool that tells you how much of currency A you need to get a specific amount of currency B

In [ ]:
# Exercise 1: Add AUD to the currency converter
# Hint: AUD rate relative to USD is approximately 1.53
# Your code here:



In [ ]:
# Exercise 2: Add negative amount validation
# Your code here:



In [ ]:
# Exercise 3: Create a "how much do I need" tool
# Example: "How many USD do I need to get 1000 EUR?"
# Your code here:

